### Step 1

Initialize the notebook, loading the configuration and importing libraries.

In [42]:
# Import the libraries required for the notebook
import asyncio

# Instatiate the config class
import sys
sys.path.append('..')
from config.notebook_config import *
config = notebook_config()
# Load config from file
config.load_config_from_file()


### Step 2

Load the data from a Word file, and store it in a data dictionary.

Replace filename by the fullpath to the Word file.

In [43]:
# Open the Word document
from docx import Document
filename = "C://Users//perocha//OneDrive//Meile//RETT//2023 Marseille//7th European Rett Syndrome Conference.docx"
if not os.path.exists(filename):
    raise FileNotFoundError(f"File not found at {filename}")
document = Document(filename)

# Initialize an empty list for sessions and a None for the current session
sessions = []
current_session = None
current_session_content = ""

# Iterate through paragraphs and group content by "Heading 2"
for paragraph in document.paragraphs:
    if paragraph.style.style_id == "Heading2":
        # If there is a current session, append it to the sessions list
        if current_session is not None:
            current_session["content"] = current_session_content
            sessions.append(current_session)
        # Start a new session with the current paragraph as the title
        current_session = {"title": paragraph.text, "content": ""}
        current_session_content = ""
    else:
        # If there is a current session, append the paragraph to its content
        if current_session is not None:
            current_session_content += paragraph.text + "\n"

# Append the last session if it exists
if current_session is not None:
    current_session["content"] = current_session_content
    sessions.append(current_session)


print(f"Found {len(sessions)} sessions")
# Print the header of each session (and the index)
for i, session in enumerate(sessions):
    print(f"{i}: {session['title']}")

# Print the content of the first session
print(f"1st session: {sessions[0]['title']}")
print(f"1st content: {sessions[0]['content']}")


Found 23 sessions
0: Health data applied to rare diseases: review of the national plan for rare diseases 3 and prospects for the national plan for rare diseases 4 in 2024. Anne-Sophie LAPOINTE (France)
1: PNDS or Rett center. Nadia BAHI-BUISSON (France)
2: Planning for the future. Rett Syndrome Europe (RSE). Rebecca JENNER (Royaume-Uni)
3: Trofinetide for the Treatment of Rett Syndrome 
4: NEAT1 dysregulation in Rett Syndrome: unveiling pathological mechanisms and therapeutic potential
5: Being a parent of an individual with rett syndrome in a lifelong perspective: a qualitative study of the parents experiences.
6: Gene discovery to clinical trials : How clinical and basic research have intersected to develop and test new therapies for Rett syndrome. Jeffrey NEUL (Etats-Unis)
7: Epilepsy and Rett. Mathieu MILH (Marseille)
8: Dysautonomia and Erratic Breathing in Rett Syndrome. Jan-Marino RAMIREZ (Etats-Unis)
9: Disturbances in breathing and sleep: a state of the art. Karen SPRUYT (Fran

### Step 3

Initiate the Semantic Kernel and fire-up with OpenAI model.

In [44]:
# Load Semantic Kernel
import semantic_kernel
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = semantic_kernel.Kernel()
print("Kernel loaded.")

deployment = config.model
endpoint = config.endpoint
api_key = config.azure_api_key
kernel.add_chat_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
print(f"Fire-up the kernel with {deployment}")

Kernel loaded.
Fire-up the kernel with gpt-35-turbo


### Step 4

Load plug-ins required to curate the congress notes.

In [45]:
plugins_directory = "../plugins-sk"

# Import the semantic functions
my_functions = kernel.import_semantic_skill_from_directory(plugins_directory, "CreateReport")

print("Plugins loaded.")

Plugins loaded.


### Step 5

Execute the "curation" process:
* First it will replace abbreviations and acronyms by their full meaning
* Then it will summarize the notes
* It will then extract the key concepts, in a report format
* Finally it will translate to Spanish

In [46]:
from semantic_kernel import ContextVariables

session_id = 12
input_str = sessions[session_id]["content"]
print(f"Input string: {input_str}")

# Review notes from the previous step, and summarize the notes
step1 = asyncio.create_task(kernel.run_async(my_functions["NoteReview"], input_str=input_str))
step1_result = await step1

print(step1_result.result)

# If the initial notes are not good enough, terminate the process
if step1_result.result == "TERMINATE PROCESS":
    print("Process terminated")
else:
    # Replace abbreviations and acronyms from the input text
    step2 = asyncio.create_task(kernel.run_async(my_functions["AbbreviationExpansion"], input_str=step1_result.result))
    step2_result = await step2
    print("Abbreviations expansion result:")
    print(step2_result.result)

    # Create the final session report, after the previous steps
    step3 = asyncio.create_task(kernel.run_async(my_functions["Summarize"], input_str=step2_result.result))
    step3_result = await step3
    print("Report result:")
    print(step3_result.result)

    # Translate to Spanish
    context = ContextVariables()
    context["language"] = "spanish"
    context["input"] = step3_result.result
    translate = asyncio.create_task(kernel.run_async(my_functions["TranslateNotes"], input_vars=context))
    translate_result = await translate
    print("Translate result:")
    print(translate_result)

Input string: Mouse model that they use was created by adrian bird few years ago. It has both male and famale mice models. They mainly use male mice because the symptoms are very robust, very expressed, but then they also need to confirm with females which is more complex but recreates human situation better. 
Did MRI for animal models looking for possible biomarkers (3 times in their lifespan). The brain defect is there before the symtoms are visible, all the regions of the brain are affected, some reagions a bit more than other but they all are affected. 
If the brain volume is reduced, why? What is going on? We don´t know exactly why, but we know that the neurons are affected. The soma is smaller, dendrites are much reduced, and synapses buttons are also much reduced. The number of the cell is the same, but the problem is that neurons are smaller, less matured. 
What about molecular features? 
Neural Precursos Stem Cells transplantation:
Experiment in vitro: Defect of neuronal struc

### Step 6

Repeat the previous step for the full document and save the document as output.

In [47]:
output_result = []

for session in sessions:
    input_str = session["content"]

    print(f"Processing session: {session['title']}")

    # Replace abbreviations and acronyms from the input text
    step1 = asyncio.create_task(kernel.run_async(my_functions["NoteReview"], input_str=input_str))
    step1_result = await step1

    if step1_result.result == "TERMINATE PROCESS":
        print("Process terminated")
        output_result.append({"title": session["title"], "content": "No notes available"})
    else:
        # Review notes from the previous step, and summarize the notes
        step2 = asyncio.create_task(kernel.run_async(my_functions["AbbreviationExpansion"], input_str=step1_result.result))
        step2_result = await step2

        # Create the final session report, after the previous steps
        step3 = asyncio.create_task(kernel.run_async(my_functions["Summarize"], input_str=step2_result.result))
        step3_result = await step3

        # Translate to Spanish
        context = ContextVariables()
        context["language"] = "spanish"
        context["input"] = step3_result.result
        translate = asyncio.create_task(kernel.run_async(my_functions["TranslateNotes"], input_vars=context))
        translate_result = await translate

        output_result.append({"title": session["title"], "content": translate_result.result})

    # Save the output to a Word document
    output_document = Document()

for session in output_result:
    output_document.add_heading(session["title"], level=1)
    output_document.add_paragraph(session["content"])

output_document.save("output.docx")

Processing session: Health data applied to rare diseases: review of the national plan for rare diseases 3 and prospects for the national plan for rare diseases 4 in 2024. Anne-Sophie LAPOINTE (France)
Processing session: PNDS or Rett center. Nadia BAHI-BUISSON (France)
Processing session: Planning for the future. Rett Syndrome Europe (RSE). Rebecca JENNER (Royaume-Uni)
Processing session: Trofinetide for the Treatment of Rett Syndrome 
Processing session: NEAT1 dysregulation in Rett Syndrome: unveiling pathological mechanisms and therapeutic potential
Processing session: Being a parent of an individual with rett syndrome in a lifelong perspective: a qualitative study of the parents experiences.
Processing session: Gene discovery to clinical trials : How clinical and basic research have intersected to develop and test new therapies for Rett syndrome. Jeffrey NEUL (Etats-Unis)
Processing session: Epilepsy and Rett. Mathieu MILH (Marseille)
Processing session: Dysautonomia and Erratic Bre